<a href="https://colab.research.google.com/github/ykitaguchi77/CorneAI/blob/main/%E5%89%8D%E5%8E%9F%E3%81%AE250%E5%95%8F_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **前原の250問　前処理**

## **スマホ画像resize**

ここはローカルでやる（ファイルサイズが大きいため）

### PNGでリサイズ (今回はjpgでやるので省略)

In [ ]:
import os
from PIL import Image
from tqdm import tqdm

# 元のディレクトリのパスを指定
original_directory = "/Users/kitaguchi/Downloads/スマホ"

# 新しいディレクトリのパスを指定
new_directory = "/Users/kitaguchi/Downloads/スマホ_resized"

# 新しいディレクトリが存在しない場合は作成
if not os.path.exists(new_directory):
    os.makedirs(new_directory)

# 元のディレクトリ内のすべてのファイルを処理
png_files = [filename for filename in os.listdir(original_directory) if filename.lower().endswith(".png")]

with tqdm(total=len(png_files), unit="file") as pbar:
    for filename in png_files:
        # 画像を開く
        image_path = os.path.join(original_directory, filename)
        with Image.open(image_path) as img:
            # 画像のサイズを取得
            width, height = img.size

            # 横幅を640pxに設定し、縦幅を比率に合わせて計算
            new_width = 640
            new_height = int(height * (new_width / width))

            # 画像をリサイズ
            resized_img = img.resize((new_width, new_height), Image.LANCZOS)

            # リサイズした画像を新しいディレクトリに保存
            new_image_path = os.path.join(new_directory, filename)
            resized_img.save(new_image_path)

        pbar.update(1)

print("リサイズが完了しました。")

100%|██████████| 240/240 [02:06<00:00,  1.90file/s]

リサイズが完了しました。


### JPGでリサイズ

In [ ]:
import os
from PIL import Image
from tqdm import tqdm

# 元のディレクトリのパスを指定
original_directory = "/Users/kitaguchi/Downloads/スマホ"
# 新しいディレクトリのパスを指定
new_directory = "/Users/kitaguchi/Downloads/スマホ_resized"

# 新しいディレクトリが存在しない場合は作成
if not os.path.exists(new_directory):
    os.makedirs(new_directory)

# 元のディレクトリ内のすべてのファイルを処理
jpg_files = [filename for filename in os.listdir(original_directory) if filename.lower().endswith(".png")]

with tqdm(total=len(jpg_files), unit="file") as pbar:
    for filename in jpg_files:
        # 画像を開く
        image_path = os.path.join(original_directory, filename)
        with Image.open(image_path) as img:
            # 画像のサイズを取得
            width, height = img.size

            # 横幅を640pxに設定し、縦幅を比率に合わせて計算
            new_width = 640
            new_height = int(height * (new_width / width))

            # 画像をリサイズ
            resized_img = img.resize((new_width, new_height), Image.LANCZOS)

            # 画像をRGBモードに変換
            rgb_img = resized_img.convert("RGB")

            # リサイズした画像を新しいディレクトリに保存
            new_filename = os.path.splitext(filename)[0] + ".jpg"  # ファイル名をjpg形式に変更
            new_image_path = os.path.join(new_directory, new_filename)
            rgb_img.save(new_image_path, "JPEG")  # JPEG形式で保存

        pbar.update(1)

print("リサイズが完了しました。")

100%|██████████| 240/240 [01:16<00:00,  3.14file/s]

リサイズが完了しました。


### 対応表にないものを削除

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

# 対応表のCSVファイルを読み込む
csv_file = "/Users/kitaguchi/Downloads/対応表2022.xlsx - Sheet1.csv"
df = pd.read_csv(csv_file)

# 画像ファイルがあるディレクトリのパス
image_dir = "/Users/kitaguchi/Downloads/スマホ_resized"

# ディレクトリ内の画像ファイルを取得
image_files = [filename for filename in os.listdir(image_dir) if filename.lower().endswith((".jpg", ".png"))]

# tqdmを使用してファイル処理の進捗を表示
for filename in tqdm(image_files, desc="Processing files"):
    # ファイル名から拡張子を除いたベース名を取得
    basename, extension = os.path.splitext(filename)

    # 対応表からimg_idがベース名と一致する行を検索
    matched_rows = df[df["img_id"] == basename]

    # 一致する行が見つかった場合
    if not matched_rows.empty:
        # 最初の一致行のbasenameを取得
        new_basename = str(matched_rows.iloc[0]["basename"])  # 文字列に変換

        # 新しいファイル名を生成（拡張子を維持）
        new_filename = new_basename + extension

        # ファイルをリネーム
        old_path = os.path.join(image_dir, filename)
        new_path = os.path.join(image_dir, new_filename)
        os.rename(old_path, new_path)

        # Matchしたファイルのメッセージを表示
        tqdm.write(f"Matched and renamed: {filename} -> {new_filename}")
    else:
        # 対応表にないファイルを削除
        file_path = os.path.join(image_dir, filename)
        os.remove(file_path)

        # 削除したファイルのメッセージを表示
        tqdm.write(f"No match found. Deleted: {filename}")

### さらにリストにないものを削除

In [ ]:
import os
import pandas as pd

# CSVファイルを読み込む
csv_file = "/Users/kitaguchi/Downloads/updated_corr_table_modified3.csv"
df = pd.read_csv(csv_file, encoding='shift_jis')

# 画像ファイルがあるディレクトリのパス
image_dir = "/Users/kitaguchi/Downloads/スマホ_resized"

# ディレクトリ内のファイルを取得
files = os.listdir(image_dir)

# CSVファイルをbasename列で辞書化
csv_dict = df.set_index('basename').to_dict(orient='index')

# 一致するベース名を検索し、一致しないファイルを削除
matching_basenames = []
for file in files:
    basename = os.path.splitext(file)[0]
    if basename in csv_dict:
        matching_basenames.append(basename)
    else:
        file_path = os.path.join(image_dir, file)
        os.remove(file_path)
        print(f"一致するベース名が見つからないため、ファイルを削除しました: {file}")

# 結果を出力
if matching_basenames:
    print("一致するベース名が見つかったファイル:")
    for basename in matching_basenames:
        print(basename)
else:
    print("一致するベース名が見つかりませんでした。")

print(f"一致するベース名が見つかったファイルの数： {len(matching_basenames)}")

## **フォトスリット画像Resize**

フォトスリットもダウンロードしてリサイズする

In [ ]:
import os
from PIL import Image
from tqdm import tqdm

# 元のディレクトリのパスを指定
original_directory = "/Users/kitaguchi/Downloads/フォトスリット"

# 新しいディレクトリのパスを指定
new_directory = "/Users/kitaguchi/Downloads/フォトスリット_resized"

# 新しいディレクトリが存在しない場合は作成
if not os.path.exists(new_directory):
    os.makedirs(new_directory)

# 元のディレクトリ内のすべてのファイルを処理
image_extensions = (".jpg", ".png", ".PNG")
image_files = [filename for filename in os.listdir(original_directory) if filename.lower().endswith(image_extensions)]

with tqdm(total=len(image_files), unit="file") as pbar:
    for filename in image_files:
        # 画像を開く
        image_path = os.path.join(original_directory, filename)
        with Image.open(image_path) as img:
            # 画像のサイズを取得
            width, height = img.size

            # 横幅を640pxに設定し、縦幅を比率に合わせて計算
            new_width = 640
            new_height = int(height * (new_width / width))

            # 画像をリサイズ
            resized_img = img.resize((new_width, new_height), Image.LANCZOS)

            # 画像をRGBモードに変換
            rgb_img = resized_img.convert("RGB")

            # リサイズした画像を新しいディレクトリに保存
            new_filename = os.path.splitext(filename)[0] + ".jpg"  # ファイル名をjpg形式に変更
            new_image_path = os.path.join(new_directory, new_filename)
            rgb_img.save(new_image_path, "JPEG")  # JPEG形式で保存

        pbar.update(1)

print("リサイズが完了しました。")

100%|██████████| 238/238 [00:16<00:00, 14.61file/s]

リサイズが完了しました。


In [ ]:
import os
import pandas as pd

# CSVファイルを読み込む
csv_file = "/Users/kitaguchi/Downloads/updated_corr_table_modified3.csv"
df = pd.read_csv(csv_file, encoding='shift_jis')

# 画像ファイルがあるディレクトリのパス
image_dir = "/Users/kitaguchi/Downloads/フォトスリット_resized"

# ディレクトリ内のファイルを取得
files = os.listdir(image_dir)

# CSVファイルをbasename列で辞書化
csv_dict = df.set_index('basename').to_dict(orient='index')

# 一致するベース名を検索し、一致しないファイルを削除
matching_basenames = []
for file in files:
    basename = os.path.splitext(file)[0]
    if basename in csv_dict:
        matching_basenames.append(basename)
    else:
        file_path = os.path.join(image_dir, file)
        os.remove(file_path)
        print(f"一致するベース名が見つからないため、ファイルを削除しました: {file}")

# 結果を出力
if matching_basenames:
    print("一致するベース名が見つかったファイル:")
    for basename in matching_basenames:
        print(basename)
else:
    print("一致するベース名が見つかりませんでした。")

print(f"一致するベース名が見つかったファイルの数： {len(matching_basenames)}")

### スマホ&スリットで欠損している画像を同定

In [ ]:
import os

dir1 = "/Users/kitaguchi/Downloads/スマホ_resized"
dir2 = "/Users/kitaguchi/Downloads/フォトスリット_resized"

# dir1とdir2のファイル一覧を取得
files1 = os.listdir(dir1)
files2 = os.listdir(dir2)

# dir1のファイルをループ
for file1 in files1:
    # 拡張子を除いたファイル名を取得
    basename1 = os.path.splitext(file1)[0]

    # dir2のファイルをループ
    found = False
    for file2 in files2:
        # 拡張子を除いたファイル名を取得
        basename2 = os.path.splitext(file2)[0]

        # basenameが一致すればfound=Trueにしてループを抜ける
        if basename1 == basename2:
            found = True
            break

    # dir2で一致するファイルが見つからなかった場合、パスを表示
    if not found:
        print(os.path.join(dir1, file1))

# dir2のファイルをループ
for file2 in files2:
    # 拡張子を除いたファイル名を取得
    basename2 = os.path.splitext(file2)[0]

    # dir1のファイルをループ
    found = False
    for file1 in files1:
        # 拡張子を除いたファイル名を取得
        basename1 = os.path.splitext(file1)[0]

        # basenameが一致すればfound=Trueにしてループを抜ける
        if basename2 == basename1:
            found = True
            break

    # dir1で一致するファイルが見つからなかった場合、パスを表示
    if not found:
        print(os.path.join(dir2, file2))

--> 不一致ファイルを削除し、updated_corr_table_modified4.csvとして保存

## ここでスマホ画像をgdridveにuploadする --> ここからはgdrive上でやる